## CONTENT
#####################
 
 1. Bali Regency data
 * from Scrapy spider, combined with Wikipedia Data and geojson file from indo.gov.co.id
 2. Indo Province Data (transittion to kawal data - march 2021)
 * kawalCovid data with Kaggle Dataset by Hendra
 3. World - Country Data
 * owid world data

# Setup

In [1]:
pip install kaggle

In [2]:
from kaggle.api.kaggle_api_extended import KaggleApi

# from keys import kaggle_api
## 1. with Kaggle Api
api =KaggleApi()
api.authenticate()

In [3]:
pip install json_lines

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import json
import json_lines


# Data Loading and Processing

###  -------------------------------------
### 1. COVID DATA BALI REGENCIES
###  -------------------------------------

From scrapy spider

In [5]:
## Converting Json-Lines File to pandas DataFrame
##########################################
json_file = r'C:\Users\ansve\Coding\Projects-WebScraping\CovidBali\CovidBali\CovidBali\daily.jl'

## Extract data from First TABLE
###############################################
data_first = []
with open(json_file, 'rb') as f:
    count = 0
    for line in json_lines.reader(f, broken=True):
        count+=1
        if count % 2 == 0: #this is the remainder operator
            for x in range(0, 10):
                data_first.append(line['data'][x])

df_first = pd.DataFrame(data_first)
df_first.head()

Error: JSON line can't be decoded. JSONDecodeError('Extra data: line 1 column 875 (char 874)',)


,0,1,2,3,4,5,6,7,8,9,10
0,1,JEMBRANA,-7,70,7,579,1,14,1,663,05 Dec 2020
1,2,TABANAN,-10,434,35,1016,0,47,25,1497,05 Dec 2020
2,3,BADUNG,-9,159,27,2299,0,51,18,2509,05 Dec 2020
3,4,DENPASAR,1,166,26,3705,0,90,27,3961,05 Dec 2020
4,5,GIANYAR,-16,234,31,1681,0,75,15,1990,05 Dec 2020


In [6]:
# df_first.tail()

In [7]:
## Column names from https://infocorona.baliprov.go.id/API/pendataan/laporan-harian-02.php = df_first and ...-01.php = df_second

columns_first = [
    'No','Kabupaten / County', 'new_treatment', 'total_treatment', 'new_recovered', 'total_recovered', 'new_deaths', 'total_deaths', 'new_cases', 'total_cases', 'Date']

df_first.columns = columns_first
# df_first.head()

In [8]:
################################################
##Extract data from SECOND TABLE
###############################################
data_second = []

with open(json_file, 'rb') as f:
    count =0
    for line in json_lines.reader(f, broken=True):
        count+= 1
        if count % 2 != 0:
            for x in range(0, 10):
                data_second.append(line['data'][x])
df_second = pd.DataFrame(data_second)

Error: JSON line can't be decoded. JSONDecodeError('Extra data: line 1 column 875 (char 874)',)


In [9]:
df_second.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12
1295,6,BANGLI,0,59,0,32,10,2169,0,0,10,2260,25 Apr 2021
1296,7,KLUNGKUNG,0,20,0,69,2,1616,0,0,2,1705,25 Apr 2021
1297,8,KARANGASEM,0,23,0,249,0,1537,0,0,0,1809,25 Apr 2021
1298,9,BULELENG,0,44,0,32,8,3401,0,0,8,3477,25 Apr 2021
1299,10,KABUPATEN LAINNYA,0,9,0,142,0,464,0,0,0,615,25 Apr 2021


In [10]:
# # Second data entry
columnName_second =['No', 'Kabupaten / County', 'new PPLN/PMI', 'total history-of-foreign-travel/ migrant-worker (PPLN/PMI)', 'new domestic-travel-history', 'total domestic-travel-history (PPDN)', 'new_local_transmission', 'total_local_transmission',	'new_other_transmission', 'total_other_transmission', 'new_cases' ,'total_cases', 'Date' ]

df_second.columns = columnName_second


In [11]:
df_second = df_second.drop(['Kabupaten / County','new_cases', 'total_cases'], axis=1)

In [12]:
df_second.tail()

,No,new PPLN/PMI,total history-of-foreign-travel/ migrant-worker (PPLN/PMI),new domestic-travel-history,total domestic-travel-history (PPDN),new_local_transmission,total_local_transmission,new_other_transmission,total_other_transmission,Date
1295,6,0,59,0,32,10,2169,0,0,25 Apr 2021
1296,7,0,20,0,69,2,1616,0,0,25 Apr 2021
1297,8,0,23,0,249,0,1537,0,0,25 Apr 2021
1298,9,0,44,0,32,8,3401,0,0,25 Apr 2021
1299,10,0,9,0,142,0,464,0,0,25 Apr 2021


In [13]:
############################
## Merge both df
#############################
df_merged = pd.merge(df_first, df_second, how= 'left', left_on=['No', "Date"], right_on=['No', "Date"])

df_bali = df_merged.copy()
# df_bali.head()

In [14]:
df = df_bali.copy()
# df.tail()

In [15]:
# delete duplicate columns and convert dtypes
df = df.loc[:,~df.columns.duplicated()]

## fill NaN in Kabubaten Lain fileds (confirmed cases from outside of bali regency but detected in Bali have no match with reference-data e.g)
################################################
df = df.fillna(0)
df[df.isna().any(axis=1)]

## convert columns data types, and capitalize Names
df['No'] = df['No'].astype(int)
df['Name_EN'] = df['Kabupaten / County'].astype(str)
df.replace("KABUPATEN LAIN", "Others", inplace=True)
df['Name_EN'] = df['Name_EN'].str.capitalize()

df.dtypes

No                                                             int32
Kabupaten / County                                            object
new_treatment                                                 object
total_treatment                                               object
new_recovered                                                 object
total_recovered                                               object
new_deaths                                                    object
total_deaths                                                  object
new_cases                                                     object
total_cases                                                   object
Date                                                          object
new PPLN/PMI                                                  object
total history-of-foreign-travel/ migrant-worker (PPLN/PMI)    object
new domestic-travel-history                                   object
total domestic-travel-history (PPD

In [16]:
## convert date to datetime 
################################

# from datetime import datetime -> datetime.strptime(date, format)
# df["Date"].apply(lambda x: parse(df['Date']))
# or
# df['Date'] = pd.to_datetime(df['Date'])
## also possible

from dateutil.parser import parse
# check if Nan
# df['Date'].isnull().values

df['Date'] = df['Date'].str.strip()
df['Date'] = df['Date'].astype(str) 
df['Date'] = pd.to_datetime(df.Date, errors='coerce')
df['Date'][0]


Timestamp('2020-12-05 00:00:00')

In [17]:
## Reference Data 
## additional data like population, density etc.

# data cleaning, processing
##########################

# add addtitional data
df_ref = pd.read_excel('Bali_reference_data.xlsx')
df_ref = df_ref.drop(columns=['https://en.wikipedia.org/wiki/Bali#cite_note-BPS2019-2', 'https://sp2010.bps.go.id/index.php/site/tabel?tid=321&wid=0', 'Source'])

## adjust regency names of both data sources before merge
##########################
df_ref['Name_EN'] = df_ref['Name_Indo'].str.capitalize()
df_ref.replace('Kota denpasar', 'Denpasar', inplace=True)
df_ref.replace('totals', 'Bali', inplace=True)
df_ref.replace('Totals', 'Bali', inplace=True)

df_ref.head()

,Name_Indo,Name_EN,Capital,area_km2,population_2000,population_2010,population_2015,HDI_2014_estimate,Gouvernor,Nr. of Sub-Districts,Nr of Keluruhan/Villages
0,Bali,Bali,NaN,"5,780.06","3,146,999","3,890,757","4,148,588",0.724 (High),NaN,NaN,NaN
1,Buleleng,Buleleng,Singaraja,"1,364.73",558.181,624.125,645.893,0.691 (Medium),Putu Agus Suradnyana,9.0,19/129
2,Jembrana,Jembrana,Negara,841.80,231.806,261.638,271.423,0.686 (Medium),I Putu Artha,5.0,10/41
3,Karangasem,Karangasem,Amlapura,839.54,360.486,396.487,408.487,0.640 (Medium),I Gusti Ayu Mas Sumatri,8.0,3/75
4,Tabanan,Tabanan,Tabanan,839.30,376.03,420.913,435.753,0.726 (High),Ni Putu Eka Wiryastuti,10.0,-/133


In [18]:
# merge both df on 'Name_EN'
df_merged = df.merge(df_ref, on="Name_EN", how='left')

In [19]:
df_merged.columns

Index(['No', 'Kabupaten / County', 'new_treatment', 'total_treatment',
       'new_recovered', 'total_recovered', 'new_deaths', 'total_deaths',
       'new_cases', 'total_cases', 'Date', 'new PPLN/PMI',
       'total history-of-foreign-travel/ migrant-worker (PPLN/PMI)',
       'new domestic-travel-history', 'total domestic-travel-history (PPDN)',
       'new_local_transmission', 'total_local_transmission',
       'new_other_transmission', 'total_other_transmission', 'Name_EN',
       'Name_Indo', 'Capital', 'area_km2', 'population_2000',
       'population_2010', 'population_2015', 'HDI_2014_estimate', 'Gouvernor',
       'Nr. of Sub-Districts', 'Nr of Keluruhan/Villages'],
      dtype='object')

In [20]:
# # convert to integers
int_numbers = ['new_treatment', 'total_treatment',
       'new_recovered', 'total_recovered', 'new_deaths', 'total_deaths',
       'new_cases', 'total_cases', 'new PPLN/PMI',
       'total domestic-travel-history (PPDN)', 
       'new domestic-travel-history', 'total domestic-travel-history (PPDN)',
       'new_local_transmission', 'total_local_transmission',
       'new_other_transmission', 'total_other_transmission', 'population_2015']
## check for duplicated columns
# print(df_merged.columns[df_merged.columns.duplicated(keep=False)])
df_merged[int_numbers] = df_merged[int_numbers].apply(pd.to_numeric)
# df_merged.tail()

In [21]:
df_merged.population_2015.dtypes

dtype('float64')

In [98]:
# ## add columns with new calculated data
#########################################

df_merged['total_cases_per_100k'] = df_merged['total_cases'] / df_merged['population_2015'] * 100
df_merged['total_cases_per_100k'] = df_merged['total_cases_per_100k'].apply(pd.to_numeric).round(2)

df_merged['total_deaths_per_100k'] = df_merged['total_deaths'] / df_merged['population_2015'] * 100
df_merged['total_deaths_per_100k'] = df_merged['total_deaths_per_100k'].apply(pd.to_numeric).round(2)

df_merged['new_cases_per_mil'] = (df_merged['new_cases'] / df_merged['population_2015'])
df_merged['new_cases_per_mil'] = ((df_merged['new_cases_per_mil'].apply(pd.to_numeric)) * 100)

df_merged['cases7'] = df_merged['new_cases'].rolling(window=7, center=False).mean()
df_merged['cases7_per_100k'] = df_merged['cases7'] / df_merged['population_2015'] * 100

df_merged['deaths7'] = df_merged['new_deaths'].rolling(window=7, center=False).mean()
df_merged['deaths7_per_100k'] = df_merged['total_deaths_per_100k'] / df_merged['population_2015'] * 100

# infection fatality ratio (IFR), which estimates this proportion of deaths among all infected individuals. The second is case fatality ratio (CFR), which estimates this proportion of deaths among identified confirmed cases. 
# see https://www.who.int/news-room/commentaries/detail/estimating-mortality-from-covid-19
df_merged['CFR'] = df_merged['total_deaths'] / df_merged['total_cases'] * 100
df_merged['CFR'] = df_merged['CFR'].round(2)

## Calculate in App after region selected and df only contains region
df_merged['growth_rate_new_cases'] = df_merged['new_cases'].pct_change(fill_method ='ffill', periods=7)


In [101]:
## Check new created values
##############################
# df_merged[['total_deaths', 'total_cases', 'total_deaths_per_100k', 'population_2015', 'total_cases_per_100k']]
# df_merged[['Name_EN', 'Date', "total_cases", 'total_deaths', 'population_2015', 'total_cases_per_100k', 'total_deaths_per_100k', ]][-20:]
# df_merged[['Name_EN', "Date", 'population_2015', 'new_cases', 'new_deaths', 'new_cases_per_mil',  'cases7_per_100k', 'new_cases_per_mil', 'population_2015', ]][-5:]

,total_deaths,total_cases,total_deaths_per_100k,population_2015,total_cases_per_100k
0,14,663,5.16,271.423,244.27
1,47,1497,10.79,435.753,343.54
2,51,2509,8.29,615.148,407.87
3,90,3961,10.24,879.098,450.58
4,75,1990,15.16,494.729,402.24
...,...,...,...,...,...
1305,91,2260,40.90,222.474,1015.85
1306,63,1705,35.88,175.573,971.11
1307,94,1809,23.01,408.487,442.85
1308,139,3477,21.52,645.893,538.32


In [25]:
## add id from geojson file
geojson_bali = 'new_bali_id.geojson'

# open new geojson (with id) file
bali_geo_id = json.load(open(geojson_bali, 'r'))

## get names of regencies
regencies = []
for regency in bali_geo_id['features']:
  regencies.append(regency['properties']['ADM2_EN'])

  # add id column in df with feature.id from geojson
bali_id_map = {}
for regency in bali_geo_id['features']:
  bali_id_map[regency['properties']['ADM2_EN']] = regency['id']
bali_id_map

## for Foreigners or Other Regency add 'id' = 0
df_merged['id'] = df_merged['Name_Indo'].apply(lambda x: bali_id_map[x] if x in bali_id_map else 0)
df_merged.head()

,No,Kabupaten / County,new_treatment,total_treatment,new_recovered,total_recovered,new_deaths,total_deaths,new_cases,total_cases,...,total_cases_per_100k,total_deaths_per_100k,new_cases_per_mil,cases7,cases7_per_100k,deaths7,deaths7_per_100k,CFR,growth_rate_new_cases,id
0,1,JEMBRANA,-7,70,7,579,1,14,1,663,...,244.27,5.16,36.842862,NaN,NaN,NaN,1.901092,2.11,NaN,5
1,2,TABANAN,-10,434,35,1016,0,47,25,1497,...,343.54,10.79,573.719515,NaN,NaN,NaN,2.476173,3.14,NaN,9
2,3,BADUNG,-9,159,27,2299,0,51,18,2509,...,407.87,8.29,292.612510,NaN,NaN,NaN,1.347643,2.03,NaN,1
3,4,DENPASAR,1,166,26,3705,0,90,27,3961,...,450.58,10.24,307.132993,NaN,NaN,NaN,1.164830,2.27,NaN,8
4,5,GIANYAR,-16,234,31,1681,0,75,15,1990,...,402.24,15.16,303.196295,NaN,NaN,NaN,3.064304,3.77,NaN,4


In [26]:
df_merged.tail()

,No,Kabupaten / County,new_treatment,total_treatment,new_recovered,total_recovered,new_deaths,total_deaths,new_cases,total_cases,...,total_cases_per_100k,total_deaths_per_100k,new_cases_per_mil,cases7,cases7_per_100k,deaths7,deaths7_per_100k,CFR,growth_rate_new_cases,id
1305,6,BANGLI,6,71,3,2098,1,91,10,2260,...,1015.85,40.90,449.490727,8.571429,3.852778,0.428571,18.384171,4.03,-0.545455,2
1306,7,KLUNGKUNG,0,37,1,1605,1,63,2,1705,...,971.11,35.88,113.912731,8.285714,4.719242,0.571429,20.435944,3.70,-0.500000,7
1307,8,KARANGASEM,-6,76,5,1639,1,94,0,1809,...,442.85,23.01,0.000000,8.285714,2.028391,0.714286,5.632982,5.20,NaN,6
1308,9,BULELENG,-6,185,14,3153,0,139,8,3477,...,538.32,21.52,123.859525,8.714286,1.349184,0.571429,3.331821,4.00,0.600000,3
1309,10,KABUPATEN LAINNYA,0,42,0,562,0,11,0,615,...,NaN,NaN,NaN,7.571429,NaN,0.571429,NaN,1.79,-1.000000,0


In [27]:
# create csv for Bali Dash App
df_merged.to_csv('bali_regency_data.csv')


In [28]:
# save a version in Kaggle Datasets (upload daily)

df_merged.to_csv(r'C:\Users\ansve\Coding\Kaggle\Datasets\Covid19_Bali\Covid19_Bali_Regencies_Data.csv')

In [29]:
###  -------------------------------------
### 2. INDONESIA COVID DATA 
###  -------------------------------------

## From Kaggel Dataset
# 1. connect to kaggle and get latest dataset


In [30]:
# downoad single file
#Signature: dataset_download_file(dataset, file_name, path=None, force=False, quiet=True)
# api.dataset_download_files('hendratno/covid19-indonesia/covid_19_indonesia_time_series_all.csv', unzip = True)
# 

In [31]:
df_indo = pd.read_csv('covid_19_indonesia_time_series_all.csv')
df_indo.tail()

,Date,Location ISO Code,Location,New Cases,New Deaths,New Recovered,New Active Cases,Total Cases,Total Deaths,Total Recovered,...,Longitude,Latitude,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,Case Fatality Rate,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths
13009,3/28/2021,ID-SG,Sulawesi Tenggara,4,0,1,3,10249,205,9593,...,122.070311,-4.124689,1.52,3888.88,0.00,77.79,2.00%,93.60%,0.31,0.00
13010,3/28/2021,ID-SA,Sulawesi Utara,5,0,4,1,15351,499,12420,...,124.521240,1.259638,1.89,5810.63,0.00,188.88,3.25%,80.91%,0.26,1.00
13011,3/28/2021,ID-SB,Sumatera Barat,106,2,74,30,31455,682,29611,...,100.465062,-0.850253,19.21,5699.15,0.36,123.57,2.17%,94.14%,0.96,2.00
13012,3/28/2021,ID-SS,Sumatera Selatan,101,3,77,21,17443,828,15450,...,104.169465,-3.216212,12.29,2122.65,0.37,100.76,4.75%,88.57%,1.20,1.50
13013,3/28/2021,ID-SU,Sumatera Utara,69,2,37,30,27120,903,23714,...,99.051964,2.191894,4.64,1823.21,0.13,60.71,3.33%,87.44%,0.86,0.67


In [32]:
## convert dates to datetime format so matching with new dataframe before merging
print(df_indo['Date'][0])
df_indo['Date'] = pd.to_datetime(df_indo['Date'])


3/1/2020


In [33]:
df_indo.columns

Index(['Date', 'Location ISO Code', 'Location', 'New Cases', 'New Deaths',
       'New Recovered', 'New Active Cases', 'Total Cases', 'Total Deaths',
       'Total Recovered', 'Total Active Cases', 'Location Level',
       'City or Regency', 'Province', 'Country', 'Continent', 'Island',
       'Time Zone', 'Special Status', 'Total Regencies', 'Total Cities',
       'Total Districts', 'Total Urban Villages', 'Total Rural Villages',
       'Area (km2)', 'Population', 'Population Density', 'Longitude',
       'Latitude', 'New Cases per Million', 'Total Cases per Million',
       'New Deaths per Million', 'Total Deaths per Million',
       'Case Fatality Rate', 'Case Recovered Rate',
       'Growth Factor of New Cases', 'Growth Factor of New Deaths'],
      dtype='object')

In [34]:
# adjust column names
columns_new = ['Date', 'Location ISO Code', 'Location', 'new_cases', 'new_deaths',
       'new_recovered', 'New Active Cases', 'total_cases', 'total_deaths',
       'total_recovered', 'Total Active Cases', 'Location Level',
       'City or Regency', 'Province', 'Country', 'Continent', 'Island',
       'Time Zone', 'Special Status', 'Total Regencies', 'Total Cities',
       'Total Districts', 'Total Urban Villages', 'Total Rural Villages',
       'Area (km2)', 'Population', 'Population Density', 'Longitude',
       'Latitude', 'New Cases per Million', 'Total Cases per Million',
       'New Deaths per Million', 'Total Deaths per Million',
       'CFR', 'Case Recovered Rate',
       'Growth Factor of New Cases', 'Growth Factor of New Deaths']

df_indo.columns = columns_new
df_indo.columns

Index(['Date', 'Location ISO Code', 'Location', 'new_cases', 'new_deaths',
       'new_recovered', 'New Active Cases', 'total_cases', 'total_deaths',
       'total_recovered', 'Total Active Cases', 'Location Level',
       'City or Regency', 'Province', 'Country', 'Continent', 'Island',
       'Time Zone', 'Special Status', 'Total Regencies', 'Total Cities',
       'Total Districts', 'Total Urban Villages', 'Total Rural Villages',
       'Area (km2)', 'Population', 'Population Density', 'Longitude',
       'Latitude', 'New Cases per Million', 'Total Cases per Million',
       'New Deaths per Million', 'Total Deaths per Million', 'CFR',
       'Case Recovered Rate', 'Growth Factor of New Cases',
       'Growth Factor of New Deaths'],
      dtype='object')

In [35]:
df_indo['Population'].iloc[1]


10846145

In [36]:
## add id from geojson for choropleth Map
#########################################
# open new geojson (with id) file
geojson_indo = 'new_indo_id.geojson'
indo_geo_id = json.load(open(geojson_indo, 'r'))


In [37]:
# indo_geo_id['features'][7]['id']

In [38]:
## ADJUST NAMES OF df_indo with NAMES OF GEOJSON ADM1_EN

# which names are different
names_df = df_indo['Location'].unique()
provinces = []
for province in indo_geo_id['features']:
  provinces.append(province['properties']['state'])

list(set(provinces) - set(names_df))

['Yogyakarta', 'Bangka-Belitung', 'Jakarta Raya', 'Irian Jaya Barat']

In [39]:
list_df = ['DKI Jakarta', 'Kalimantan Utara', 'Daerah Istimewa Yogyakarta', 'Papua Barat','Kepulauan Bangka Belitung']

list(set(provinces) - set(names_df))
list1= ['Bangka-Belitung', 'Jakarta Raya', 'Yogyakarta', 'Irian Jaya Barat']


In [40]:
# adjust Names of provnices in Indo

df_indo = df_indo.replace("DKI Jakarta", 'Jakarta Raya')
df_indo = df_indo.replace("Papua Barat", 'Irian Jaya Barat')
df_indo = df_indo.replace("Daerah Istimewa Yogyakarta", 'Yogyakarta')
df_indo = df_indo.replace("Kepulauan Bangka Belitung", 'Bangka-Belitung')

In [41]:
df_indo.columns

Index(['Date', 'Location ISO Code', 'Location', 'new_cases', 'new_deaths',
       'new_recovered', 'New Active Cases', 'total_cases', 'total_deaths',
       'total_recovered', 'Total Active Cases', 'Location Level',
       'City or Regency', 'Province', 'Country', 'Continent', 'Island',
       'Time Zone', 'Special Status', 'Total Regencies', 'Total Cities',
       'Total Districts', 'Total Urban Villages', 'Total Rural Villages',
       'Area (km2)', 'Population', 'Population Density', 'Longitude',
       'Latitude', 'New Cases per Million', 'Total Cases per Million',
       'New Deaths per Million', 'Total Deaths per Million', 'CFR',
       'Case Recovered Rate', 'Growth Factor of New Cases',
       'Growth Factor of New Deaths'],
      dtype='object')

In [42]:
df_indo.Location.unique()

array(['Jakarta Raya', 'Indonesia', 'Jawa Barat', 'Riau', 'Banten',
       'Sulawesi Tenggara', 'Kalimantan Timur', 'Yogyakarta',
       'Sumatera Utara', 'Jawa Tengah', 'Jawa Timur', 'Sulawesi Selatan',
       'Jambi', 'Maluku', 'Papua', 'Maluku Utara', 'Aceh',
       'Kalimantan Tengah', 'Lampung', 'Sulawesi Tengah',
       'Sumatera Barat', 'Irian Jaya Barat', 'Kalimantan Utara',
       'Sulawesi Barat', 'Sumatera Selatan', 'Bali', 'Kalimantan Barat',
       'Kalimantan Selatan', 'Bangka-Belitung', 'Kepulauan Riau',
       'Bengkulu', 'Nusa Tenggara Barat', 'Sulawesi Utara',
       'Nusa Tenggara Timur', 'Gorontalo'], dtype=object)

In [43]:
provinces = []
for province in indo_geo_id['features']:
  provinces.append(province['properties']['state'])

# add id column in df with feature.id from geojson
indo_id_map = {}
for province in indo_geo_id['features']:
  indo_id_map[province['properties']['state']] = province['id']
# indo_id_map

# ## for Foreigners or Other Regency add 'id' = 0
df_indo['id'] = df_indo['Location'].apply(lambda x: indo_id_map[x] if x in indo_id_map else 0)

df_indo.tail(2)


,Date,Location ISO Code,Location,new_cases,new_deaths,new_recovered,New Active Cases,total_cases,total_deaths,total_recovered,...,Latitude,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,CFR,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths,id
13012,2021-03-28,ID-SS,Sumatera Selatan,101,3,77,21,17443,828,15450,...,-3.216212,12.29,2122.65,0.37,100.76,4.75%,88.57%,1.20,1.50,33
13013,2021-03-28,ID-SU,Sumatera Utara,69,2,37,30,27120,903,23714,...,2.191894,4.64,1823.21,0.13,60.71,3.33%,87.44%,0.86,0.67,4


## KawalCovid19 Data

## get latest Province Data from 
https://docs.google.com/spreadsheets/d/1ma1T9hWbec1pXlwZ89WakRk-OfVUQZsOCFl4FwZxzVw/edit#gid=2052139453

* process and clean
* merge with df_indo
* update daily


In [44]:
df = pd.read_excel('kawalcovid19.xlsx', sheet_name="Timeline")


In [45]:
# df.head()

In [46]:
## transform excel sheet to timeseries table (columns: Data, Province, total_cases, daily_cases, active_cases etc.)
case_type = ['Total Kasus', 'Kasus Harian', 'Kasus Aktif', 'Sembuh', 'Sembuh Harian', 'Meninggal Dunia', 'Meninggal Dunia Harian', 'Meninggal Dunia Harian']

In [47]:
# extract df for each case-type
##################################

# replace all NaN with 0
# delete empty rows
# adjust header
# extract for each Province, add Province Name and merge, and sort by date

df_totalKasus = df.iloc[:df.loc[df['Total Kasus'].str.contains('Kasus Harian', na=False)].index[0]]
df_totalKasus = df_totalKasus.rename(columns={'Total Kasus': "Date"})
df_totalKasus= df_totalKasus.fillna(0)
df_totalKasus.set_index('Date', inplace=True)
df_totalKasus= df_totalKasus[:-1]
df_totalKasus = df_totalKasus.drop(['?', 'Unnamed: 36'], axis=1)

df_kasusHarian = df.iloc[df.loc[df['Total Kasus'].str.contains('Kasus Harian', na=False)].index[0]:df.loc[df['Total Kasus'].str.contains('Kasus Aktif', na=False)].index[0]]
df_kasusHarian = df_kasusHarian.rename(columns={'Total Kasus': "Date"})
df_kasusHarian= df_kasusHarian.fillna(0)
df_kasusHarian.set_index('Date', inplace=True)
df_kasusHarian= df_kasusHarian[1:-1]
df_kasusHarian = df_kasusHarian.drop(['?', 'Unnamed: 36'], axis=1)

df_kasusAktif = df.iloc[df.loc[df['Total Kasus'].str.contains('Kasus Aktif', na=False)].index[0]:df.loc[df['Total Kasus'].str.contains('Sembuh', na=False)].index[0]]
df_kasusAktif = df_kasusAktif.rename(columns={'Total Kasus': "Date"})
df_kasusAktif= df_kasusAktif.fillna(0)
df_kasusAktif.set_index('Date', inplace=True)
df_kasusAktif= df_kasusAktif[1:-1]
df_kasusAktif = df_kasusAktif.drop(['?', 'Unnamed: 36'], axis=1)

df_sembuh = df.iloc[df.loc[df['Total Kasus'].str.contains('Sembuh', na=False)].index[0]:df.loc[df['Total Kasus'].str.contains('Sembuh Harian', na=False)].index[0]]
df_sembuh = df_sembuh.rename(columns={'Total Kasus': "Date"})
df_sembuh= df_sembuh.fillna(0)
df_sembuh.set_index('Date', inplace=True)
df_sembuh= df_sembuh[1:-1]
df_sembuh = df_sembuh.drop(['?', 'Unnamed: 36'], axis=1)

df_sembuhHarian = df.iloc[df.loc[df['Total Kasus'].str.contains('Sembuh Harian', na=False)].index[0]:df.loc[df['Total Kasus'].str.contains('Meninggal Dunia', na=False)].index[0]]
df_sembuhHarian = df_sembuhHarian.rename(columns={'Total Kasus': "Date"})
df_sembuhHarian= df_sembuhHarian.fillna(0)
df_sembuhHarian.set_index('Date', inplace=True)
df_sembuhHarian= df_sembuhHarian[1:-1]
df_sembuhHarian = df_sembuhHarian.drop(['?', 'Unnamed: 36'], axis=1)

df_meninggalDunia = df.iloc[df.loc[df['Total Kasus'].str.contains('Meninggal Dunia', na=False)].index[0]:df.loc[df['Total Kasus'].str.contains('Meninggal Dunia Harian', na=False)].index[0]]
df_meninggalDunia = df_meninggalDunia.rename(columns={'Total Kasus': "Date"})
df_meninggalDunia= df_meninggalDunia.fillna(0)
df_meninggalDunia.set_index('Date', inplace=True)
df_meninggalDunia= df_meninggalDunia[1:-1]
df_meninggalDunia = df_meninggalDunia.drop(['?', 'Unnamed: 36'], axis=1)

df_meninggalDuniaHarian = df.iloc[df.loc[df['Total Kasus'].str.contains('Meninggal Dunia Harian', na=False)].index[0]:df.loc[df['Total Kasus'].str.contains('Meninggal Dunia Harian', na=False)].index[1]]
df_meninggalDuniaHarian = df_meninggalDuniaHarian.rename(columns={'Total Kasus': "Date"})
df_meninggalDuniaHarian= df_meninggalDuniaHarian.fillna(0)
df_meninggalDuniaHarian.set_index('Date', inplace=True)
df_meninggalDuniaHarian= df_meninggalDuniaHarian[1:-3]
df_meninggalDuniaHarian = df_meninggalDuniaHarian.drop(['?', 'Unnamed: 36'], axis=1)

dfs = [df_totalKasus, df_kasusHarian, df_kasusAktif, df_sembuh, df_sembuhHarian, df_meninggalDunia, df_meninggalDuniaHarian]
dfs_names = ["df_totalKasus", "df_kasusHarian", "df_kasusAktif", "df_sembuh","df_sembuhHarian", "df_meninggalDunia", "df_meninggalDuniaHarian"]
dfs_type = ["totalKasus", "kasusHarian", "kasusAktif", "sembuh","sembuhHarian", "meninggalDunia", "meninggalDuniaHarian"]

In [48]:
column_names = df_totalKasus.columns
# column_names

Index(['Aceh', 'Bali', 'Banten', 'Babel', 'Bengkulu', 'DIY', 'Jakarta',
       'Jambi', 'Jabar', 'Jateng', 'Jatim', 'Kalbar', 'Kaltim', 'Kalteng',
       'Kalsel', 'Kaltara', 'Kep Riau', 'NTB', 'Sumsel', 'Sumbar', 'Sulut',
       'Sumut', 'Sultra', 'Sulsel', 'Sulteng', 'Lampung', 'Riau', 'Malut',
       'Maluku', 'Papbar', 'Papua', 'Sulbar', 'NTT', 'Gorontalo'],
      dtype='object')

In [49]:
## First Dataframe - Total Cases per Province

df_totKasus = pd.DataFrame({'Province': [0], "Total Kasus":[0]})
for i, province in enumerate(column_names):
    case_numbers = df_totalKasus[province]
    df_x = pd.DataFrame({'Province': case_numbers, "Total Kasus": case_numbers })
    df_x["Province"] = column_names[i]
    # df_x['Total Kasus'] = df_totalKasus[province]
    if i == 0:
        df_totKasus = df_x
    else:
        df_totKasus = df_totKasus.append(df_x)

In [50]:
print(df_totKasus.shape)
df_totKasus1 = df_totKasus.reset_index()
# df_totKasus1.head()

(13736, 2)


,Date,Province,Total Kasus
0,2020-03-18,Aceh,0
1,2020-03-19,Aceh,0
2,2020-03-20,Aceh,0
3,2020-03-21,Aceh,0
4,2020-03-22,Aceh,0


In [51]:
## since memory error split dataframes (only with join)
## learning: if not merge on index -> use .merge() 2) reset_index() get datetime out of index first

dfs1 =dfs[1:]
df_total = pd.DataFrame({"Date": [0], 'Province': [0], dfs_type[0]:[0]})

for index, df in enumerate(dfs1):
    count = index + 1
    df_name = dfs_names[index+1]+'Clean'
    df_name = pd.DataFrame({"Date": [0], 'Province': [0], dfs_type[count]:[0]})
    for i, province in enumerate(column_names):
        case_numbers = df[province]
        df_x = pd.DataFrame({'Province': case_numbers, dfs_type[count]: case_numbers })
        df_x["Province"] = column_names[i]
        if i == 0:
            df_name = df_x
        else:
            df_name = df_name.append(df_x)

    df_name = df_name.reset_index()
    if index == 0:
        df_total = df_name
    else:
        df_total = pd.merge(df_total, df_name, on=['Date', 'Province'])   

In [52]:
## sort_dates
# check if datetime format correct
print(df_total['Date'].iloc[0])
df_total = df_total.sort_values(by="Date")
# df_total.head()

2020-03-21 00:00:00


,Date,Province,kasusHarian,kasusAktif,sembuh,sembuhHarian,meninggalDunia,meninggalDuniaHarian
0,2020-03-21,Aceh,0,0,0,0,0,0
9950,2020-03-21,Lampung,0,1,0,0,0,0
9552,2020-03-21,Sulteng,0,0,0,0,0,0
9154,2020-03-21,Sulsel,2,2,0,0,0,0
8756,2020-03-21,Sultra,0,3,0,0,0,0


In [53]:
# merge with total Kasus df from before
df_total = pd.merge(df_total, df_totKasus1, on=['Date', 'Province'])
# df_total.tail() 

,Date,Province,kasusHarian,kasusAktif,sembuh,sembuhHarian,meninggalDunia,meninggalDuniaHarian,Total Kasus
13527,2021-04-23,Maluku,8,349,7042,11,116,0,7507
13528,2021-04-23,Papbar,24,371,8292,18,144,0,8807
13529,2021-04-23,Papua,8,8923,11149,3,207,0,20279
13530,2021-04-23,Kaltara,19,557,10885,14,186,0,11628
13531,2021-04-23,Gorontalo,2,194,4985,2,161,0,5340


In [54]:
# adjust Province Names
df_total['Province'].unique()

array(['Aceh', 'Lampung', 'Sulteng', 'Sulsel', 'Sultra', 'Sumut', 'NTT',
       'Sulut', 'Sumbar', 'Sumsel', 'NTB', 'Kep Riau', 'Kaltara', 'Riau',
       'Kalsel', 'Kaltim', 'Kalbar', 'Gorontalo', 'Jatim', 'Jateng',
       'Jabar', 'Jambi', 'Jakarta', 'DIY', 'Bengkulu', 'Babel', 'Banten',
       'Kalteng', 'Malut', 'Bali', 'Maluku', 'Papua', 'Sulbar', 'Papbar'],
      dtype=object)

In [55]:
# adjust Province Names
indo_slang_prov_names = ['Aceh', 'Bali', 'Maluku', 'Sulbar', 'Malut', 'Banten', 'Riau',
       'Lampung', 'Babel', 'Sulteng', 'Sulsel', 'Bengkulu', 'Sultra',
       'Sumut', 'DIY', 'Papbar', 'Sulut', 'NTT', 'Jakarta', 'Sumsel',
       'NTB', 'Jambi', 'Kep Riau', 'Jabar', 'Kaltara', 'Kalsel',
       'Gorontalo', 'Jateng', 'Kalteng', 'Kaltim', 'Jatim', 'Sumbar',
       'Papua', 'Kalbar']

# from previous Indo-province dataset
correct_prov_names = ['Jakarta Raya', 'Indonesia', 'Jawa Barat', 'Riau', 'Banten',
       'Sulawesi Tenggara', 'Kalimantan Timur', 'Yogyakarta',
       'Sumatera Utara', 'Jawa Tengah', 'Jawa Timur', 'Sulawesi Selatan',
       'Maluku', 'Papua', 'Maluku Utara', 'Aceh', 'Kalimantan Tengah',
       'Lampung', 'Sulawesi Tengah', 'Sumatera Barat', 'Irian Jaya Barat',
       'Kalimantan Utara', 'Sulawesi Barat', 'Sumatera Selatan', 'Bali',
       'Kalimantan Barat', 'Jambi', 'Kalimantan Selatan',
       'Bangka-Belitung', 'Kepulauan Riau', 'Bengkulu',
       'Nusa Tenggara Barat', 'Nusa Tenggara Timur',
       'Gorontalo']


In [56]:
# print names which are different in both dfs and make same
print(list(set(indo_slang_prov_names) - set(correct_prov_names)))

# adjust column names
df_total = df_total.replace({
    'Sultra': 'Sulawesi Utara',
    "Jakarta": 'Jakarta Raya',
    "Kalteng": 'Kalimantan Tengah',
    'Sulsel': 'Sulawesi Selatan',
    'Sulut': 'Sulawesi Utara',
    'Kaltim': "Kalimantan Timur", 'Kalsel': 'Kalimantan Selatan','Malut': 'Maluku Utara','Sumsel': 'Sumatera Barat','NTT': 'Nusa Tenggara Timur','Kalbar': 'Kalimantan Barat','Jateng': 'Jawa Tengah','Kep Riau': 'Kepulauan Riau','Kaltara': 'Kalimantan Utara','Sulbar': 'Sulawesi Barat','Babel': 'Bangka-Belitung','Jatim': "Jawa Timur",'Sumbar': 'Sumatera Barat',"Sumut": 'Sumatera Utara','Jabar': 'Jawa Barat','NTB': 'Nusa Tenggara Barat','DIY': 'Yogyakarta','Sulteng': 'Sulawesi Tengah','Papbar': 'Papua','Jakarta': 'Jakarta Raya'})


['Kalsel', 'NTB', 'Jatim', 'Sulteng', 'Sulut', 'Sumbar', 'Sumsel', 'Sulsel', 'DIY', 'Sumut', 'Sulbar', 'Kalbar', 'Papbar', 'Jakarta', 'Jateng', 'Kalteng', 'Kaltim', 'NTT', 'Sultra', 'Jabar', 'Kep Riau', 'Babel', 'Malut', 'Kaltara']


In [57]:
df_total.columns

Index(['Date', 'Province', 'kasusHarian', 'kasusAktif', 'sembuh',
       'sembuhHarian', 'meninggalDunia', 'meninggalDuniaHarian',
       'Total Kasus'],
      dtype='object')

In [58]:
#adjust column names
new_col_names = ['Date', 'Province', 'new_cases', 'active_cases', 'total_recovered', 'new_recovered', 'total_deaths', 'new_deaths', 'total_cases']
df_total.columns = new_col_names
df_total.tail()

,Date,Province,new_cases,active_cases,total_recovered,new_recovered,total_deaths,new_deaths,total_cases
13527,2021-04-23,Maluku,8,349,7042,11,116,0,7507
13528,2021-04-23,Papua,24,371,8292,18,144,0,8807
13529,2021-04-23,Papua,8,8923,11149,3,207,0,20279
13530,2021-04-23,Kalimantan Utara,19,557,10885,14,186,0,11628
13531,2021-04-23,Gorontalo,2,194,4985,2,161,0,5340


In [59]:
# Get previous Indo_df with selected columns and merge with df_total Indo Province 
# use only selected columns
cols = ['Location', 'Location Level', 'City or Regency', 'Country', 'Continent',
       'Island', 'Time Zone', 'Special Status', 'Total Regencies',
       'Total Cities', 'Total Districts', 'Total Urban Villages',
       'Total Rural Villages', 'Area (km2)', 'Population',
       'Population Density', 'Longitude', 'Latitude','id']
df_indo_small = df_indo[cols]
df_indo_small.tail()

,Location,Location Level,City or Regency,Country,Continent,Island,Time Zone,Special Status,Total Regencies,Total Cities,Total Districts,Total Urban Villages,Total Rural Villages,Area (km2),Population,Population Density,Longitude,Latitude,id
13009,Sulawesi Tenggara,Province,NaN,Indonesia,Asia,Sulawesi,UTC+08:00,NaN,15,2.0,219,377.0,1911.0,38068,2635461,69.23,122.070311,-4.124689,23
13010,Sulawesi Utara,Province,NaN,Indonesia,Asia,Sulawesi,UTC+08:00,NaN,11,4.0,171,332.0,1507.0,13892,2641884,190.17,124.521240,1.259638,22
13011,Sumatera Barat,Province,NaN,Indonesia,Asia,Sumatera,UTC+07:00,NaN,12,7.0,179,230.0,928.0,42013,5519245,131.37,100.465062,-0.850253,2
13012,Sumatera Selatan,Province,NaN,Indonesia,Asia,Sumatera,UTC+07:00,NaN,13,4.0,241,387.0,2853.0,91592,8217551,89.72,104.169465,-3.216212,33
13013,Sumatera Utara,Province,NaN,Indonesia,Asia,Sumatera,UTC+07:00,NaN,25,8.0,450,693.0,5417.0,72981,14874889,203.82,99.051964,2.191894,4


In [60]:
print(df_indo_small.shape, df_total.shape)

(13014, 19) (13532, 9)


In [61]:
df_indo_small = df_indo_small.drop_duplicates(subset='Location')
df_indo_small.shape

(35, 19)

In [62]:
# check if same Province Names
print(list(set(df_total.Province.unique()) - set(df_indo_small.Location.unique())))

[]


In [63]:
df_indo_full = pd.merge(df_total, df_indo_small, left_on='Province', right_on='Location')
df_indo_full = df_indo_full.sort_values(by='Date')
df_indo_full.tail()

,Date,Province,new_cases,active_cases,total_recovered,new_recovered,total_deaths,new_deaths,total_cases,Location,...,Total Cities,Total Districts,Total Urban Villages,Total Rural Villages,Area (km2),Population,Population Density,Longitude,Latitude,id
11541,2021-04-23,Maluku Utara,3,190,4092,6,120,0,4402,Maluku Utara,...,2.0,116,118.0,1063.0,31983,1307803,40.89,127.539107,0.212037,14
11939,2021-04-23,Bali,138,1374,41124,147,1293,10,43791,Bali,...,1.0,57,80.0,636.0,5780,4216171,729.43,115.131714,-8.369472,24
12337,2021-04-23,Maluku,8,349,7042,11,116,0,7507,Maluku,...,2.0,118,35.0,1198.0,46914,1847097,39.37,129.576792,-3.192572,13
6367,2021-04-23,Kalimantan Timur,188,2017,63954,181,1611,3,67582,Kalimantan Timur,...,3.0,103,197.0,841.0,129067,3552191,27.52,116.468440,0.453858,10
13531,2021-04-23,Sulawesi Barat,2,84,5232,4,117,0,5433,Sulawesi Barat,...,NaN,69,73.0,575.0,16787,1559984,92.93,119.345019,-2.461746,32


In [64]:
# ## add additional metrics
#########################################

df_indo_full['total_cases_per_100k'] = df_indo_full['total_cases'] / df_indo_full['Population'] * 100000
df_indo_full['total_cases_per_100k'] = df_indo_full['total_cases_per_100k'].apply(pd.to_numeric)

df_indo_full['total_deaths_per_100k'] = df_indo_full['total_deaths'] / df_indo_full['Population'] * 100000
df_indo_full['total_deaths_per_100k'] = df_indo_full['total_deaths_per_100k'].apply(pd.to_numeric)

df_indo_full['new_cases_per_mil'] = df_indo_full['new_cases'] / df_indo_full['Population'] *1000000
df_indo_full['new_cases_per_mil'] = df_indo_full['new_cases_per_mil'].apply(pd.to_numeric)

df_indo_full['cases7'] = df_indo_full['new_cases'].rolling(window=7, center=False).mean()
df_indo_full['cases7_per_100k'] = df_indo_full['cases7'] / df_indo_full['Population'] * 100000
df_indo_full['cases7_per_100k'] = df_indo_full['cases7_per_100k'].apply(pd.to_numeric)

df_indo_full['deaths7'] = df_indo_full['new_deaths'].rolling(window=7, center=False).mean()
df_indo_full['deaths7_per_100k'] = df_indo_full['total_deaths_per_100k'].rolling(window=7, center=False).mean()
df_indo_full['deaths7_per_100k'] = df_indo_full['deaths7_per_100k'].apply(pd.to_numeric)

df_indo_full['CFR'] = df_indo_full['total_deaths'] / df_indo_full['total_cases'] * 100
df_indo_full['CFR'] = df_indo_full['CFR'].round(2)

## Or Calculate in App after region selected and df only contains region?
df_indo_full['growth_rate_new_cases'] = df_indo_full['new_cases'].pct_change(fill_method ='ffill', periods=7)

In [102]:
df_indo_full.columns

Index(['Date', 'Province', 'new_cases', 'active_cases', 'total_recovered',
       'new_recovered', 'total_deaths', 'new_deaths', 'total_cases',
       'Location', 'Location Level', 'City or Regency', 'Country', 'Continent',
       'Island', 'Time Zone', 'Special Status', 'Total Regencies',
       'Total Cities', 'Total Districts', 'Total Urban Villages',
       'Total Rural Villages', 'Area (km2)', 'Population',
       'Population Density', 'Longitude', 'Latitude', 'id',
       'total_cases_per_100k', 'total_deaths_per_100k', 'new_cases_per_mil',
       'cases7', 'cases7_per_100k', 'deaths7', 'deaths7_per_100k', 'CFR',
       'growth_rate_new_cases', 'Name_EN'],
      dtype='object')

In [103]:
# Check new created values

# df_indo_full[['Date', "Province",'new_cases', 'total_deaths', 'new_deaths', 'total_cases', 'Population', 'total_cases_per_100k', 'total_deaths_per_100k', 'new_cases_per_mil', 'cases7_per_100k', 'deaths7', 'deaths7_per_100k',]].tail()

,Date,Province,new_cases,total_deaths,new_deaths,total_cases,Population,total_cases_per_100k,total_deaths_per_100k,new_cases_per_mil,cases7_per_100k,deaths7,deaths7_per_100k
11541,2021-04-23,Maluku Utara,3,120,0,4402,1307803,336.595038,9.175694,2.293923,18.690015,4.000000,20.809283
11939,2021-04-23,Bali,138,1293,10,43791,4216171,1038.643831,30.667637,32.731120,3.269724,2.857143,16.539505
12337,2021-04-23,Maluku,8,116,0,7507,1847097,406.421536,6.280125,4.331121,6.465744,2.142857,13.769844
6367,2021-04-23,Kalimantan Timur,188,1611,3,67582,3552191,1902.544092,45.352291,52.925082,3.756233,2.285714,19.048466
13531,2021-04-23,Sulawesi Barat,2,117,0,5433,1559984,348.272803,7.500077,1.282064,5.467089,2.142857,18.163054


In [65]:
df_indo_full['Name_EN'] = df_indo_full['Province']


In [67]:
df_indo_full.columns

Index(['Date', 'Province', 'new_cases', 'active_cases', 'total_recovered',
       'new_recovered', 'total_deaths', 'new_deaths', 'total_cases',
       'Location', 'Location Level', 'City or Regency', 'Country', 'Continent',
       'Island', 'Time Zone', 'Special Status', 'Total Regencies',
       'Total Cities', 'Total Districts', 'Total Urban Villages',
       'Total Rural Villages', 'Area (km2)', 'Population',
       'Population Density', 'Longitude', 'Latitude', 'id',
       'total_cases_per_100k', 'total_deaths_per_100k', 'new_cases_per_mil',
       'cases7', 'cases7_per_100k', 'deaths7', 'deaths7_per_100k', 'CFR',
       'growth_rate_new_cases', 'Name_EN'],
      dtype='object')

In [68]:
df_indo_full.to_csv('indo_province_data.csv')

# 3. World Data 
Comparison Data

https://github.com/owid/covid-19-data/blob/master/public/data/owid-covid-data.csv

url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"


In [69]:
url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"
df_world = pd.read_csv(url)


In [70]:
df_world.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511


In [104]:
# pd.unique(df_world['location'])

In [72]:
df_world.columns

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'new_tests', 'total_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'new_vaccinations',
       'new_vaccinations_smoothed', 'total_vaccinations_per_hun

In [73]:
# df_world['location'].unique()

In [74]:
# relevant countries :
rel_countries = ["Germany", 'Europe', 'World', 'International', 'Indonesia',  'United Kingdom', 'Australia', 'France', "Italy", 'United States']

In [75]:
# CFR, total cases per 100k and total deaths, growth_rate, vaccination
## add columns for total cases/deaths per 100k
df_world['Date'] = df_world['date'] 
df_world['CFR'] = df_world['total_deaths'] / df_world['total_cases'] * 100
df_world['total_cases_per_100k'] = df_world['total_cases_per_million'] / 10
df_world['total_deaths_per_100k'] = df_world['total_deaths_per_million'] / 10
df_world['new_cases_per_mil'] = df_world['new_cases_per_million']


In [76]:
df_germany = df_world[df_world['location'] == 'Germany']
df_germany.head()
df_germany['total_cases_per_100k'].iloc[-1]
# df_germany['total_cases_per_million'].iloc[-1]

3928.3099

In [77]:
df_world.to_csv("world_data.csv")